# Introduction

Welcome to the P1 Workspace with Group 14:
Will Mitchell
Trevor Gordon


Below are some prelimnary Helper methods that benifit the program as a whole with the starting code parameters

In [203]:
# Parameters
GROUP_ID = 'Group14'
ALGORITHM = 'bt'
PUZZLE_TYPE = 'easy'
PUZZLE_PATH = 'puzzles/Med-P1.txt'

In [16]:
def parse_matrix(file_path : str):
  with open(file_path, mode="r") as file:
      new_matrix = []
      for line in file:
        new_line_array = line.replace("\n","").replace("\ufeff","").split(",")
        new_matrix.append(new_line_array)


  file.close()
  new_matrix[0][0] = str(new_matrix[0][0])[-1:]
  return new_matrix

parse_matrix(PUZZLE_PATH)



[['?', '?', '8', '?', '5', '6', '?', '?', '?'],
 ['7', '?', '4', '?', '?', '?', '6', '1', '9'],
 ['?', '?', '?', '?', '?', '?', '8', '5', '?'],
 ['6', '?', '7', '?', '2', '9', '5', '?', '?'],
 ['?', '?', '9', '?', '6', '?', '1', '?', '?'],
 ['?', '?', '2', '3', '1', '?', '9', '?', '4'],
 ['?', '3', '5', '?', '?', '?', '?', '?', '?'],
 ['4', '2', '1', '?', '?', '?', '3', '?', '6'],
 ['?', '?', '?', '8', '3', '?', '4', '?', '?']]

In [17]:
def get_square_values(sudoku_state, row, col):
    #top left box
    if col <= 2 and row <= 2:
        col_indexes = [0,1,2]
        row_indexes = [0,1,2]

    #top middle box
    elif (2 < col <= 5) and (row <= 2):
        col_indexes = [3,4,5]
        row_indexes = [0,1,2]

    #top right box
    elif (5 < col <= 8) and (row <= 2):
        col_indexes = [6,7,8]
        row_indexes = [0,1,2]

    #middle left box
    elif (col <= 2) and (2 < row <= 5):
        col_indexes = [0,1,2]
        row_indexes = [3,4,5]

    #middle middle box
    elif (2 < col <= 5) and (2 < row <= 5):
        col_indexes = [3,4,5]
        row_indexes = [3,4,5]

    #middle right box
    elif (5 < col <= 8) and (2 < row <= 5):
        col_indexes = [6,7,8]
        row_indexes = [3,4,5]

    #bottom left box
    elif (col <= 2) and (row > 5):
        col_indexes = [0,1,2]
        row_indexes = [6,7,8]

    #bottom middle box
    elif (2 < col <= 5) and (row > 5):
        col_indexes = [3,4,5]
        row_indexes = [6,7,8]

    #bottom right box
    elif (5 < col <= 8) and (row > 5):
        col_indexes = [6,7,8]
        row_indexes = [6,7,8]

    my_set = set()
    for i in row_indexes:
            for j in col_indexes:
                if sudoku_state[i][j] != '?':
                      my_set.add(sudoku_state[i][j])
    return my_set



# print(get_square_values(parse_matrix(PUZZLE_PATH), 6, 1))



{'3', '5', '2', '1', '4'}


In [204]:
def is_available(sudoku_state, row, col, num):
    my_list = get_unavailable_nums(sudoku_state,row,col)
    if (str(num) in my_list):
        return False
    else:
        return True


def solver_arc(sudoku_state, row, col):
    if row == 8 and col == 9:
        return True

    # If last column of the row go to the next row
    if col == 9:
        row += 1
        col = 0

    # If spot is already filled then go to next
    if sudoku_state[row][col] != '?':
        return solver_arc(sudoku_state, row, col+1)

    for num in range(1, 10):
        # If num is available in the current spot
        if is_available(sudoku_state, row, col, num):
            sudoku_state[row][col] = str(num)
            if solver_arc(sudoku_state, row, col+1):
                return True
            sudoku_state[row][col] = '?'
    return False

def solveSudoku(sudoku_state): #starts recursive solving
    solver_arc(sudoku_state, 0, 0)
    for line in sudoku_state:
        print(line)

solveSudoku(parse_matrix(PUZZLE_PATH))

['2', '5', '3', '9', '8', '4', '7', '1', '6']
['9', '8', '4', '1', '7', '6', '3', '2', '5']
['6', '1', '7', '3', '2', '5', '8', '4', '9']
['5', '4', '1', '7', '3', '8', '9', '6', '2']
['3', '9', '2', '6', '4', '1', '5', '8', '7']
['8', '7', '6', '5', '9', '2', '4', '3', '1']
['7', '2', '5', '4', '1', '3', '6', '9', '8']
['4', '6', '8', '2', '5', '9', '1', '7', '3']
['1', '3', '9', '8', '6', '7', '2', '5', '4']


In [165]:
class ga_state_node():
    def __init__(self, raw_matrix):
        self.matrix = raw_matrix
        self.num_conflicts = [-1] #this will track actually the history of conflicts, and push new conflict values each time its processed per generation
        self.hardcoded_value_map = self.process_hardcoded_values_map()
        #we want to immediately be able to know which indexes are hardcoded or not for a basic constraint

    def process_conflicts(self):
        self.num_conflicts.push(1)

    def process_hardcoded_values_map(self):
        return ((0,0), (0,1), (1,0), (1,1))
        #map of our indexes

    def fill_random_values(self):
        self.matrix = [] #fills question marks with random values


# Helper Functions


In [19]:
def get_row_values(sudoku_state, row, col):
  my_set = set(sudoku_state[row])
  return my_set

# print(get_row_values(parse_matrix(PUZZLE_PATH),0,0))

{'6', '8', '?', '5'}


In [166]:
def get_col_values(sudoku_state, row, col):
  my_set = set()
  for i in range(9):
      my_set.add(sudoku_state[i][col])

  return my_set

print(get_col_values(parse_matrix(PUZZLE_PATH),0,0))

{'6', '4', '?', '7'}


In [182]:
def get_unavailable_nums(sudoku_state, row, col):
    unavail_row = get_row_values(sudoku_state,row, col)
    unavail_col = get_col_values(sudoku_state,row, col)
    unavail_square = get_square_values(sudoku_state, row, col)
    my_set = unavail_row.union(unavail_col, unavail_square)
    return my_set

get_unavailable_nums(parse_matrix(PUZZLE_PATH),0,0)

KeyboardInterrupt: 